### 2.基础优化算法

#### 2.1梯度下降

  - 挑选初始值$\mathbf{w_0}$
  - 重复迭代参数t=1,2,3；$\mathbf{w}_t = \mathbf{w}_{t-1} - \eta \frac{\partial{L}}{\partial{\mathbf{w}_{t-1}}}$
    - 沿梯度反方向，将减小损失函数值
    - $\eta$学习率：步长的超参数
  - 选择学习率：
    - 不能太小，因为计算梯度很贵，不能算太多；
    - 也不能太大，会震荡。

#### 2.2小批量随机梯度下降

  - 在整个训练集上算梯度太贵；
  - 可以随机采样b个样本$i_1,i_2,...,i_b$来近似代替整个训练集上的梯度：$\frac{1}{b}\sum_{i\in I_b}{L(\mathbf{X}_i, y_i, \mathbf{W})}$；
  - b是批量大小，另一个重要的超参数：
    - 不能太小，不具有代表性；
    - 也不能太大，浪费计算，失去了本身的意义。

### 3.矢量化加速

希望对整个mini batch进行处理，将计算转为矩阵运算，避免在python中编写高代价的循环。

In [6]:
import math
import time
import numpy as np
import torch

n = 10000
a = torch.ones(n)
b = torch.ones(n)

In [7]:
class Timer:  #@save
    """记录多次运行时间"""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """启动计时器"""
        self.tik = time.time()

    def stop(self):
        """停止计时器并将时间记录在列表中"""
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        """返回平均时间"""
        return sum(self.times) / len(self.times)

    def sum(self):
        """返回时间总和"""
        return sum(self.times)

    def cumsum(self):
        """返回累计时间"""
        return np.array(self.times).cumsum().tolist()

In [8]:
c = torch.zeros(n)
timer = Timer()
for i in range(n):
    c[i] = a[i] + b[i]
f'{timer.stop():.5f} sec'

'0.10408 sec'

In [9]:
timer.start()
d = a + b
f'{timer.stop():.5f} sec'

'0.00075 sec'

重载运算符的矩阵运算具有明显的优势。

### 3.正态分布与均方损失
通过对噪声分布的假设来解读均方损失目标函数。

均方误差损失函数（简称均方损失）可以用于线性回归的一个原因是： 我们假设了观测中包含噪声，其中噪声服从均值为0的正态分布。
$$
y = \mathbf{w}^T\mathbf{x} + b + \epsilon, ~ \epsilon \sim \mathcal{N}(0, \sigma)
$$
通过给定的$\mathbf{x}$观测到特定$y$的似然（likelihood）：$P(y|\mathbf{x}) = \frac{1}{\sqrt{2\pi \sigma^2}} exp\left(-\frac{1}{2\sigma^2}(y - \mathbf{w}^T\mathbf{x} - b)^2\right)$

极大似然，最小化负对数似然：$-logP(\mathbf{y}|\mathbf{X}) = \sum_{i=1}^n \frac{1}{2}log(2\pi \sigma^2) + \frac{1}{2\sigma^2}(y^{(i)} - \mathbf{w}^T\mathbf{x}^{(i)} - b)^2$

### 4.从线性回归到深度网络

每个输入都与每个输出（在本例中只有一个输出）相连， 我们将这种变换称为全连接层（fully-connected layer）或称为稠密层（dense layer）。

### 5.小结

  - 机器学习模型中的关键要素是训练数据、损失函数、优化算法，还有模型本身。
  - 矢量化使数学表达上更简洁，同时运行的更快。
  - 最小化目标函数和执行极大似然估计等价。
  - 线性回归模型也是一个简单的神经网络

### 6.练习

假定控制附加噪声$\epsilon$的噪声模型是指数分布。也就是说，$p(\epsilon) = \frac{1}{2}exp(-|\epsilon|)$
  - 写出模型$−logP(\mathbf{y}∣\mathbf{X})$下数据的负对数似然：$\sum_{i=1}^n log(2) + |y^{(i)} - \mathbf{w}^T\mathbf{x}^{(i)} - b|$
  - 你能写出解析解吗？不能
  - 提出一种随机梯度下降算法来解决这个问题。哪里可能出错？（提示：当我们不断更新参数时，在驻点附近会发生什么情况）你能解决这个问题吗？
    - 对比上面的高斯噪声采用的mean square error(MSE)，此处使用mean absolute error(MAE)作为损失函数。
    - MAE在零点附近不可导，故在驻点附近仍然可能产生很大的梯度，导致剧烈波动难以收敛。
    - 一般可以采用smooth L1-loss代替L1-loss，取得更稳定的收敛结果。